# TCN time series forecasting

Time series forecasting with a **temporal convolutional network** (TCN) architecture is demonstrated in this notebook. The forecasting models are trained with noisy data which is artificially generated for that purpose. The same bivariate sine/cosine time series from the LSTM companion notebook is considered. The performance of the trained TCN can then be easily compared against the LSTM model.

In [ ]:
%load_ext autoreload
%autoreload 2
%matplotlib inline

import sys
sys.path.append('..')

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import torch
import torch.nn as nn
from torch.utils.data import DataLoader

from ts_utils import (
    make_sine_cosine,
    SlidingWindows,
    CausalConv,
    TCN,
    train
)

In [ ]:
np.random.seed(12345)
_ = torch.manual_seed(54321)

## Data generation

As in the other example, we start by synthetically generating time series data as follows.

In [ ]:
num_steps = 2000
max_length = 100.
noise_level = 0.1
val_size = 0.2

train_data, val_data = make_sine_cosine(
    num_steps=num_steps,
    max_length=max_length,
    noise_level=noise_level,
    val_size=val_size
)

print('Train data shape:', train_data.shape)
print('Val. data shape:', val_data.shape)

In [ ]:
fig, ax = plt.subplots(figsize=(6, 4))
ax.plot(np.arange(len(train_data)) + 1, train_data[:,0], alpha=0.7, label='sine data')
ax.plot(np.arange(len(train_data)) + 1, train_data[:,1], alpha=0.7, label='cosine data')
ax.set(xlabel='x', ylabel='y')
ax.set_xlim((0, len(train_data)))
ax.legend(loc='lower left')
ax.grid(visible=True, which='both', color='lightgray', linestyle='-')
ax.set_axisbelow(True)
fig.tight_layout()

## Data set/loader

Following this, the datasets and loaders for the training and validation sets are initialized, respectively.

In [ ]:
window_size = 64

train_set = SlidingWindows(
    train_data,
    window_size,
    mode='shift',
    time_last=True
)

val_set = SlidingWindows(
    val_data,
    window_size,
    mode='shift',
    time_last=True
)

print('No. train points:', len(train_set))
print('No. val. points:', len(val_set))

In [ ]:
batch_size = 16

train_loader = DataLoader(
    train_set,
    batch_size=batch_size,
    shuffle=True,
    drop_last=True
)

val_loader = DataLoader(
    val_set,
    batch_size=batch_size,
    shuffle=False,
    drop_last=True
)

print('No. train batches:', len(train_loader))
print('No. val. batches:', len(val_loader))

In [ ]:
X_batch, y_batch = next(iter(train_loader))
print('Input shape:', X_batch.shape)
print('Target shape:', y_batch.shape)

## Model training

A simple TCN based on so-called "causal" convolutions is implemented. It is trained with the same loss function and optimization algorithm as before.

In [ ]:
model = nn.Sequential(
    CausalConv(2, 4, kernel_size=3),
    nn.Tanh(),
    CausalConv(4, 4, kernel_size=3, dilation=2),
    nn.Tanh(),
    CausalConv(4, 2, kernel_size=3, dilation=4)
)

tcn = TCN(model)
print('No. weights:', tcn.num_trainable)

In [ ]:
criterion = nn.MSELoss()
optimizer = torch.optim.Adam(tcn.parameters(), lr=0.001)

In [ ]:
# TODO: Turn off left padding for training only on actual non-padded data
# loss = criterion(y_pred, y_batch)

train(
    tcn,
    criterion=criterion,
    optimizer=optimizer,
    num_epochs=100,
    train_loader=train_loader,
    val_loader=val_loader
)

## Test predictions

As a last step, sequential model predictions that start from the first validation set interval are plotted.

In [ ]:
seq = torch.as_tensor(val_data[:window_size])  # (time, features)
seq = seq.T.unsqueeze(0)  # (batch=1, features, time)

tcn.eval()
with torch.no_grad():
    preds = tcn.forecast(
        seq,
        steps=len(val_data) - window_size
    )

print('Pred. shape:', preds.shape)

In [ ]:
fig, ax = plt.subplots(figsize=(6, 4))

ax.plot(
    np.arange(window_size) + 1,
    val_data[:window_size,0],
    color=plt.cm.Dark2(0), alpha=0.7,
    label='test sine'
)

ax.plot(
    np.arange(window_size) + 1,
    val_data[:window_size,1],
    color=plt.cm.Dark2(1), alpha=0.7,
    label='test cosine'
)

ax.plot(
    np.arange(window_size, len(val_data)) + 1,
    preds[0,0,:],
    color=plt.cm.Dark2(0), alpha=0.7, linestyle='--',
    label='sine forecast'
)

ax.plot(
    np.arange(window_size, len(val_data)) + 1,
    preds[0,1,:],
    color=plt.cm.Dark2(1), alpha=0.7, linestyle='--',
    label='cosine forecast'
)

ax.set(xlabel='x', ylabel='y')
ax.set_xlim((0, len(val_data)))
ax.legend(loc='lower left')

ax.grid(visible=True, which='both', color='lightgray', linestyle='-')
ax.set_axisbelow(True)
fig.tight_layout()